In [83]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as T

import pickle


In [84]:
# Device configuration

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('device:', device)

device: cuda


In [85]:
from IPython.display import HTML, display


# Custom IPython progress bar for training
class ProgressMonitor(object):

    tmpl = """
        <table style="width: 100%;">
            <tbody>
                <tr>
                    <td style="width: 30%;">
                     <b>Loss: {loss:0.4f}</b> &nbsp&nbsp&nbsp {value} / {length}
                    </td>
                    <td style="width: 70%;">
                        <progress value='{value}' max='{length}', style='width: 100%'>{value}</progress>
                    </td>
                </tr>
            </tbody>
        </table>
        """

    def __init__(self, length):
        self.length = length
        self.count = 0
        self.display = display(self.html(0, 0), display_id=True)

    def html(self, count, loss):
        return HTML(self.tmpl.format(length=self.length, value=count, loss=loss))

    def update(self, count, loss):
        self.count += count
        self.display.update(self.html(self.count, loss))

In [86]:
transform_train = T.Compose( [T.RandomCrop(32, padding=4), T.ToTensor(), T.Normalize( (0.5, 0.5, 0.5), (0.5, 0.5, 0.5) )] )
transform_test = T.Compose( [T.ToTensor(), T.Normalize( (0.5, 0.5, 0.5), (0.5, 0.5, 0.5) )] )

train_set = torchvision.datasets.CIFAR10('./data', train=True, download=True, transform=transform_train )
test_set = torchvision.datasets.CIFAR10('./data', train=False, download=True, transform=transform_test )

classes = train_set.classes

Files already downloaded and verified
Files already downloaded and verified


In [87]:
print(train_set.data.shape)
print(test_set.data.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [88]:
import torch.nn as nn

class SimpleCNN(nn.Module):

    def __init__(self):
        super().__init__()

        self.conv_layers = nn.Sequential(
            #### 입력 이미지 크기 32x32x3
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),

            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),

            nn.MaxPool2d(2),

            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),

            nn.MaxPool2d(2),
        )

        self.fc_layers = nn.Sequential(
            nn.Linear(128 * 8 * 8, 500),  # 입력 크기를 128 * 8 * 8로 수정
            nn.ReLU(),
            nn.Linear(500, 10),  # 클래스 개수에 맞게 출력 크기를 10으로 변경
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1) # flatten
        x = self.fc_layers(x)
        return x

In [89]:
temp = SimpleCNN()
output = torch.randn(7, 3, 32, 32)

print(temp(output).size())

torch.Size([7, 10])


In [100]:
# 실습 2- Resnet 구현(선택)
# Hint- layer를 지나간 뒤 input을 더해주어야 한다, stride말고 maxpool로 size 줄여도 괜찮습니다.

import torch
import torch.nn as nn

class ResNet(nn.Module):
    def __init__(self):
        super().__init__()

        # 첫 번째 Convolutional 레이어 (입력 이미지 크기: 224x224x3)
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)  # Batch Normalization
        self.relu1 = nn.ReLU()         # ReLU Activation

        # 두 번째 Convolutional 레이어 (출력 크기: 112x112x64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1)
        self.bn2 = nn.BatchNorm2d(128)  # Batch Normalization
        self.relu2 = nn.ReLU()          # ReLU Activation

        # 세 번째 Convolutional 레이어 (출력 크기: 56x56x128)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1)
        self.bn3 = nn.BatchNorm2d(256)  # Batch Normalization
        self.relu3 = nn.ReLU()          # ReLU Activation

        # 네 번째 Convolutional 레이어 (출력 크기: 28x28x256)
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, stride=2, padding=1)
        self.bn4 = nn.BatchNorm2d(512)  # Batch Normalization
        self.relu4 = nn.ReLU()          # ReLU Activation

        # 다섯 번째 Convolutional 레이어 (출력 크기: 14x14x512)
        self.conv5 = nn.Conv2d(512, 1024, kernel_size=3, stride=2, padding=1)
        self.bn5 = nn.BatchNorm2d(1024)  # Batch Normalization
        self.relu5 = nn.ReLU()           # ReLU Activation

        # Adaptive Average Pooling 레이어 (출력 크기: 1x1x1024)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        # Fully Connected 레이어 (출력 크기: 10)
        self.fc = nn.Linear(1024, 10)

    def forward(self, x):
        # 입력 이미지를 각 레이어를 통과

        x = self.conv1(x)    # 첫 번째 Convolutional 레이어 적용
        x = self.bn1(x)      # Batch Normalization
        x = self.relu1(x)    # ReLU Activation
        x = self.avgpool(x)  # Adaptive Average Pooling

        x = self.conv2(x)    # 두 번째 Convolutional 레이어 적용
        x = self.bn2(x)      # Batch Normalization
        x = self.relu2(x)    # ReLU Activation
        x = self.avgpool(x)  # Adaptive Average Pooling

        x = self.conv3(x)    # 세 번째 Convolutional 레이어 적용
        x = self.bn3(x)      # Batch Normalization
        x = self.relu3(x)    # ReLU Activation
        x = self.avgpool(x)  # Adaptive Average Pooling

        x = self.conv4(x)    # 네 번째 Convolutional 레이어 적용
        x = self.bn4(x)      # Batch Normalization
        x = self.relu4(x)    # ReLU Activation
        x = self.avgpool(x)  # Adaptive Average Pooling

        x = self.conv5(x)    # 다섯 번째 Convolutional 레이어 적용
        x = self.bn5(x)      # Batch Normalization
        x = self.relu5(x)    # ReLU Activation
        x = self.avgpool(x)  # Adaptive Average Pooling

        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc(x)  # Fully Connected 레이어 적용

        return x

In [101]:
# Resnet 모델 테스트
# 텐서의 사이즈가 (7, 10)이 나오면 성공

temp = ResNet()
output = temp(torch.randn(7, 3, 32, 32))

print(output.size())

torch.Size([7, 10])


In [102]:
batch_size = 128 # 배치 사이즈
learning_rate = 0.01 # 학습률
num_epochs = 30 # 에폭 수

In [103]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False)

In [104]:
# 원하는 모델을 돌려보세요

model = SimpleCNN()

model.to(device)

SimpleCNN(
  (conv_layers): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU()
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc_layers): Sequential(
    (0): Linear(in_features=8192, out_features=500, bias=True)
    (1): ReLU()
    (2): Linear(in_features=500, out_features=10, bias=True)
  )
)

In [105]:
# Loss Function
criterion = nn.CrossEntropyLoss()
# optimizer 선정
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [106]:
from statistics import mean

def train(optimizer, model, num_epochs=10, first_epoch=1):

    criterion = nn.CrossEntropyLoss()

    train_losses = []
    test_losses = []

    for epoch in range(first_epoch, first_epoch + num_epochs):
        print('Epoch', epoch)

        # train phase
        model.train()

        # create a progress bar
        progress = ProgressMonitor(length=len(train_set))

        # keep track of predictions
        correct_train = 0

        batch_losses = []

        for batch, targets in train_loader:

            # Move the training data to the GPU
            batch = batch.to(device)
            targets = targets.to(device)

            # clear previous gradient computation
            optimizer.zero_grad()

            # forward propagation
            outputs = model(batch)

            # calculate the loss
            loss = criterion(outputs, targets)

            # backpropagate to compute gradients
            loss.backward()

            # update model weights
            optimizer.step()

            batch_losses.append(loss.item())

            # accumulate correct count
            _, preds = torch.max(outputs, 1)
            correct_train += torch.sum(preds == targets.data)

            # update progress bar
            progress.update(batch.shape[0], mean(batch_losses) )


        train_losses.append( mean(batch_losses))


        # test phase
        model.eval()

        y_pred = []

        correct_test = 0

        # We don't need gradients for test, so wrap in
        # no_grad to save memory
        with torch.no_grad():

            for batch, targets in test_loader:

                # Move the training batch to the GPU
                batch = batch.to(device)
                targets = targets.to(device)

                # forward propagation
                outputs = model(batch)

                # calculate the loss
                loss = criterion(outputs, targets)

                # save predictions
                y_pred.extend( outputs.argmax(dim=1).cpu().numpy() )

                # accumulate correct count
                _, preds = torch.max(outputs, 1)
                correct_test += torch.sum(preds == targets.data)


        # Calculate accuracy
        train_acc = correct_train.item() / train_set.data.shape[0]
        test_acc = correct_test.item() / test_set.data.shape[0]

        print('Training accuracy: {:.2f}%'.format(float(train_acc) * 100))
        print('Test accuracy: {:.2f}%\n'.format(float(test_acc) * 100))


    return train_losses, test_losses, y_pred

In [107]:
#@title
train_losses, test_losses, y_pred = train(optimizer, model, num_epochs=num_epochs)

Epoch 1


Loss: 3.4604 50000 / 50000,50000


Training accuracy: 17.21%
Test accuracy: 23.05%

Epoch 2


Loss: 1.8455 50000 / 50000,50000


Training accuracy: 25.84%
Test accuracy: 27.68%

Epoch 3


...

Training accuracy: 78.03%
Test accuracy: 76.28%

Epoch 29


Loss: 0.6153 50000 / 50000,50000


Training accuracy: 78.69%
Test accuracy: 75.56%

Epoch 30


Loss: 0.6213 50000 / 50000,50000


Training accuracy: 78.46%
Test accuracy: 76.29%



여기까지 진행하고! ipynb 파일 제출해주세요.